In [ ]:
import os
import pandas as pd
from sqlalchemy import inspect
import numpy as np
import pandas as pd
import psycopg
from sqlalchemy import create_engine
import dotenv
import os
dotenv.load_dotenv()

POSTGRES_PASSWORD = os.getenv('POSTGRES_PASSWORD')

dbserver = psycopg.connect(
    dbname="arxiv",
    host="localhost",
    port="5432",
    user="postgres",
    password=POSTGRES_PASSWORD
)
dbserver.autocommit = True

dbms = 'postgresql'
package = 'psycopg'
user = 'postgres'
password = POSTGRES_PASSWORD
host = 'localhost'
port = '5432'
db = 'arxiv'

engine = create_engine(f'{dbms}+{package}://{user}:{password}@{host}:{port}/{db}')
engine

# output folder
out_dir = "../data"
os.makedirs(out_dir, exist_ok=True)

# get all table names
inspector = inspect(engine)
tables = inspector.get_table_names()

for table in tables:

    chunks = pd.read_sql(
        f'SELECT * FROM "{table}"',
        engine,
        chunksize=100_000
    )

    out_path = f"{out_dir}/{table}.csv"
    first = True

    for chunk in chunks:
        chunk.to_csv(
            out_path,
            mode="w" if first else "a",
            header=first,
            index=False
        )
        first = False

print("All tables exported.")

All tables exported
